In [114]:
import os 
from dotenv import load_dotenv
load_dotenv()
from autogen import UserProxyAgent, AssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor
from autogen import GroupChat
from autogen import GroupChatManager

#llm configuration
llm_config = {"model": 'gpt-3.5-turbo-0125', 'api_key': os.environ['OPENAI_API_KEY']}


In [116]:
# executor = LocalCommandLineCodeExecutor(
#     timeout=60,
#     work_dir="content/current",

In [117]:
def human_input_mode(msg):
    return msg.get("content") is not None and "TERMINATE" in msg["content"]
try:
    user_proxy = UserProxyAgent(
        name='user_proxy',
        is_termination_msg=human_input_mode,
        human_input_mode= "ALWAYS",
        max_consecutive_auto_reply=5,
        code_execution_config= {
         "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
        }
    )
except KeyError as e:
    print(f"KeyError: {e}")

In [118]:
goal_assessment_agent = AssistantAgent(
    'Goal_assessment_agent',
    llm_config=llm_config,
    system_message= """Role:  Assess the user's fitness goals and current fitness level.
Responsibilities: 
Ask limited and relevant questions to determine user goals. Don't ask for too much detail.
Evaluate user current fitness level through a series of questions or initial assessments.
Example: "How many days per week can you dedicate to working out?"
"""
)

In [119]:
workout_planner_agent = AssistantAgent(
    "Workout_planner",
    llm_config= llm_config,
    system_message= """Role: Creates personalized workout plans based on user goals and preferences.
Responsibilities: 
Generate workout routines tailored to user goals (e.g., cardio for weight loss, strength training for muscle gain).
Adapt plans based on user constraints (e.g., available equipment, workout duration).
Example: "Here's a 4-week workout plan focusing on strength training and cardio."
"""
)

In [120]:
nutritional_guidance_agent = AssistantAgent(
    'Nutritionist',
    llm_config= llm_config,
    system_message= """Role: Provides dietary recommendations to complement the workout plan.
Responsibilities:
Ask the user whether he is a vegetarian or non_veg first.
Suggest dietary changes and meal plans that align with fitness goals.
Offer tips on nutrition and supplementation.
Example: "Here are some high-protein meal suggestions to support muscle gain."
"""
)

In [121]:
groupchat = GroupChat(
    agents=[user_proxy, workout_planner_agent, goal_assessment_agent, nutritional_guidance_agent],
    messages=[],
    max_round=10,
)

In [122]:
manager = GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


In [123]:
# testing 

In [124]:
# def state_transition(last_speaker, groupchat):
#     messages = groupchat.messages

#     if last_speaker is goal_assessment_agent:
#         # init -> retrieve
#         return user_proxy
#     if last_speaker is workout_planner_agent:
#         return user_proxy


In [125]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message= "I'm soon joining a Gym and want to lose weight and gain muscle",
    )

user_proxy (to chat_manager):

I'm soon joining a Gym and want to lose weight and gain muscle

--------------------------------------------------------------------------------

Next speaker: Goal_assessment_agent

Goal_assessment_agent (to chat_manager):

Great to hear that you're starting at the gym! To better help you achieve your goals, could you please share with me how many days per week you can dedicate to working out? 



--------------------------------------------------------------------------------

Next speaker: Workout_planner

Workout_planner (to chat_manager):

Based on your goal to lose weight and gain muscle, I can create a personalized 4-week workout plan for you. The plan will include a combination of cardio exercises to aid in weight loss and strength training exercises to help you gain muscle. Since you're just starting at the gym, we'll keep things simple and effective. The workout plan will consist of 4 days a week with a focus on full-body workouts to achieve you